In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

submission = pd.read_csv('data/sample_submission.csv')

In [2]:
# 원핫 인코딩
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)
train_data, train_target = train_one.drop('INVC_CONT',axis=1), train_one['INVC_CONT']

In [3]:
from tpot import TPOTRegressor

tpot_params = dict({
    'generations':10,
    'population_size':100,
    'subsample':1,
    'verbosity':2,
    'random_state':42,
    'n_jobs':-1,
    'early_stop':100,
    'use_dask':True,
    'warm_start':True,
    'memory':'auto',
    'periodic_checkpoint_folder':'./tpot_checkpoint/'
})

In [4]:
tpot = TPOTRegressor(**tpot_params)
for i in range(100):
    tpot.fit(train_data, train_target)
    tpot.export(f'tpot_pipeline_new_{i}.py')
    results = tpot.predict(test_one)
    submission['INVC_CONT'] = results
    submission.to_csv(f'submission/autosk_v{i}.csv', index=False)


Generation 1 - Current best internal CV score: -30.221347188203964

Generation 2 - Current best internal CV score: -29.152448076062747

Generation 3 - Current best internal CV score: -29.152448076062747

Generation 4 - Current best internal CV score: -29.152448076062747

Generation 5 - Current best internal CV score: -29.152448076062747

Generation 6 - Current best internal CV score: -29.01344328442915

Best pipeline: XGBRegressor(SelectFwe(input_matrix, alpha=0.042), learning_rate=0.1, max_depth=4, min_child_weight=10, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.5, verbosity=0)

Generation 1 - Current best internal CV score: -29.01344328442915

Generation 2 - Current best internal CV score: -29.00447709733575

Generation 3 - Current best internal CV score: -29.00447709733575

Generation 4 - Current best internal CV score: -29.00447709733575
Optimization Progress:  49%|████▉     | 542/1100 [59:17<1:10:10,  7.54s/pipeline]